## This is the three-linear layer model that was originally shown to us during our meeting.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data_utils
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error as MAE, mean_squared_error as MSE, mean_absolute_percentage_error as MAPE

In [2]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

def weights_init_uniform(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        # m.weight.data.uniform_(0.0,1.0)
        m.weight.data.normal_(0.0,1.0)
        m.bias.data.fill_(0)

def weights_init(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight, gain=torch.nn.init.calculate_gain("linear"))
        m.bias.data.fill_(0)

def MAPELoss(output, target):
    loss = (torch.abs(output - target) / torch.abs(target)).mean()
    return loss

# # convert a df to tensor to be used in pytorch
# def df_to_tensor(df):
#     device = get_device()
#     return torch.from_numpy(df.values).float().to(device)

In [3]:
batch_size = 2048
n_epochs = 1000
learning_rate = 0.001

In [4]:
data_path = "/data/workspace_files/"
vols = np.load(data_path + "12_12_sample_lognormal_vol.npy")
names = ["S", "T", "V_atm", "Beta", "Rho", "Volvol", "K"]

multiindex = pd.MultiIndex.from_product([range(i) for i in vols.shape],
                                        names=names
                                       )
full_df = pd.DataFrame(vols.reshape((-1,1)), index=multiindex, columns=["Lognormal_vol"]).reset_index()
print(full_df.shape)

# get features:
data_ranges = {'S': np.linspace(0.005+0.0, 0.07+0.03, num=12),
               'T': np.linspace(0.5, 20., num=5),
               'V_atm': np.linspace(0.001, 0.015, num=3),
               'Beta': np.linspace(0.1, 0.7, num=2),
               'Rho': np.linspace(-0.4, 0.4, num=3),
               'Volvol': np.linspace(0.0001, 0.5, num=5),
               'K': np.linspace(0.005+0.0, 0.07+0.03, num=12)
              }

for key in data_ranges.keys():
    full_df[key] = data_ranges[key][full_df[key]]

test_df = full_df.sample(frac=0.6, replace=False, random_state=1)
train_df = full_df.drop(test_df.index) # 19440 for 30%

train_target = torch.tensor(train_df['Lognormal_vol'].values.astype(np.float32))
train_features = torch.tensor(train_df.drop('Lognormal_vol', axis = 1).values.astype(np.float32)) 
train_tensor = data_utils.TensorDataset(train_features, train_target)
train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = False)

test_target = torch.tensor(test_df['Lognormal_vol'].values.astype(np.float32))
test_features = torch.tensor(test_df.drop('Lognormal_vol', axis = 1).values.astype(np.float32)) 

(64800, 8)


## MAPELoss

In [5]:
def create_model():
    model = torch.nn.Sequential(nn.Linear(7, 16),
                            nn.Linear(16,32),
                            nn.Linear(32, 64),
                            nn.Sigmoid(),
                            nn.Linear(64,10),
                            nn.Tanh(),
                            nn.Linear(10,1),
                            nn.Sigmoid()
                            )
    return model

model_params = {
    'train_loader': train_loader,
    'test_features': test_features,
    'test_target': test_target,
    'create_model': create_model,
    'n_epochs': n_epochs,
    'criterion': MAPELoss,
    'init_fn': weights_init
}

In [6]:
# Stability Analysis

def run_n(n, model_params):
    # model_params['model'] # do it just in case
    runs_mae = []
    runs_mse = []
    runs_mape = []
    for run in range(n):
        print(f"run {run}")
        model = create_model()
        model.apply(model_params['init_fn'])
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        for epoch in range(model_params['n_epochs']):  # loop over the dataset multiple times
            running_loss = 0.0
            # print(type(running_loss))
            for i, data in enumerate(model_params['train_loader'], 0):
                # get the inputs; data is a list of [inputs, labels]
                inputs, labels = data

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward + backward + optimize
                outputs = model(inputs)
                loss = model_params['criterion'](np.squeeze(outputs), labels)

                loss.backward()
                optimizer.step()
                
                # print statistics
                running_loss += loss.item()
            if epoch % 10 == 9:
                print(f"epoch {epoch} out of {model_params['n_epochs']} loss: {running_loss}")
        # print(f'Finished Training run {run}')
        pred = model(model_params['test_features'])
        runs_mae.append(MAE(np.squeeze(pred.cpu().detach().numpy()), model_params['test_target']))
        runs_mse.append(MSE(np.squeeze(pred.cpu().detach().numpy()), model_params['test_target']))
        runs_mape.append(MAPE(np.squeeze(pred.cpu().detach().numpy()), model_params['test_target']))
        
    print(f"mae mean: {np.mean(runs_mae)} std: {np.std(runs_mae)}")
    print(f"mse mean: {np.mean(runs_mse)} std: {np.std(runs_mse)}")
    print(f"mape mean: {np.mean(runs_mape)} std: {np.std(runs_mape)}")

In [8]:
run_n(5, model_params)

run 0
epoch 9 out of 1000 loss: 12.014709234237671
epoch 19 out of 1000 loss: 9.592508673667908
epoch 29 out of 1000 loss: 7.234126269817352
epoch 39 out of 1000 loss: 6.452080935239792
epoch 49 out of 1000 loss: 6.101460695266724
epoch 59 out of 1000 loss: 5.779362052679062
epoch 69 out of 1000 loss: 2.3583804294466972
epoch 79 out of 1000 loss: 1.3025216311216354
epoch 89 out of 1000 loss: 1.1423590071499348
epoch 99 out of 1000 loss: 0.9773812629282475
epoch 109 out of 1000 loss: 1.0189076084643602
epoch 119 out of 1000 loss: 0.7992177307605743
epoch 129 out of 1000 loss: 0.6552054230123758
epoch 139 out of 1000 loss: 0.7859694128856063
epoch 149 out of 1000 loss: 0.5434000706300139
epoch 159 out of 1000 loss: 0.6618283363059163
epoch 169 out of 1000 loss: 0.5252871382981539
epoch 179 out of 1000 loss: 0.446321826428175
epoch 189 out of 1000 loss: 0.4682282358407974
epoch 199 out of 1000 loss: 0.7152353739365935
epoch 209 out of 1000 loss: 0.46171247586607933
epoch 219 out of 1000 l

## MSELoss (Original Loss Function)

In [9]:
def create_model():
    model = torch.nn.Sequential(nn.Linear(7, 16),
                            nn.Linear(16,32),
                            nn.Linear(32, 64),
                            nn.Sigmoid(),
                            nn.Linear(64,10),
                            nn.Tanh(),
                            nn.Linear(10,1),
                            nn.Sigmoid()
                            )
    return model

model_params = {
    'train_loader': train_loader,
    'test_features': test_features,
    'test_target': test_target,
    'create_model': create_model,
    'n_epochs': n_epochs,
    'criterion': nn.MSELoss(),
    'init_fn': weights_init_uniform
}

run_n(5, model_params)

run 0
epoch 9 out of 100 loss: 0.1449949226807803
epoch 19 out of 100 loss: 0.12046138982987031
epoch 29 out of 100 loss: 0.11575134488521144
epoch 39 out of 100 loss: 0.11217760978615843
epoch 49 out of 100 loss: 0.09994395257672295
epoch 59 out of 100 loss: 0.09769393512397073
epoch 69 out of 100 loss: 0.09634557861136273
epoch 79 out of 100 loss: 0.09599288564641029
epoch 89 out of 100 loss: 0.0957610290060984
epoch 99 out of 100 loss: 0.09556683435221203
run 1
epoch 9 out of 100 loss: 0.14744093123590574
epoch 19 out of 100 loss: 0.12152450327994302
epoch 29 out of 100 loss: 0.11054960888577625
epoch 39 out of 100 loss: 0.10614472102315631
epoch 49 out of 100 loss: 0.10311789468687493
epoch 59 out of 100 loss: 0.09714487944438588
epoch 69 out of 100 loss: 0.09638750302838162
epoch 79 out of 100 loss: 0.09567401972890366
epoch 89 out of 100 loss: 0.09543857791868504
epoch 99 out of 100 loss: 0.09528265755943721
run 2
epoch 9 out of 100 loss: 0.12627828409313224
epoch 19 out of 100 l